# Physical Object: Pump

The `Pump` class models a simple, controllable pump. It is an on/off device that provides a constant flow rate as long as the head difference it is pumping against is not greater than its maximum rated head.

### State Variables
- `status` (int): The current status of the pump (1 for On, 0 for Off).
- `outflow` (float): The flow rate being produced by the pump (m^3/s).
- `power_draw_kw` (float): The electrical power being consumed by the pump (kW).

### Parameters
- `max_flow_rate` (float): The flow rate of the pump when it is on and operating within its head limit (m^3/s).
- `max_head` (float): The maximum head difference the pump can overcome (m).
- `power_consumption_kw` (float): The power consumed by the pump when it is running (kW).

## Simulation Example

The following simulation demonstrates the pump's behavior. We will:
1. Turn the pump on.
2. Increase the downstream water level until the pump can no longer overcome the head difference, causing it to shut off.
3. Turn the pump off manually.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from swp.simulation_identification.physical_objects.pump import Pump
from swp.core.interfaces import State, Parameters

# 1. Define initial state and parameters
initial_state = State(status=0, outflow=0, power_draw_kw=0)
parameters = Parameters(
    max_flow_rate=10.0,
    max_head=20.0,
    power_consumption_kw=75.0
)

# 2. Create a Pump instance
pump = Pump(name="main_pump", initial_state=initial_state, parameters=parameters)

# 3. Simulation settings
dt = 1.0
simulation_duration = 100
num_steps = int(simulation_duration / dt)
history = []
downstream_heads = []
control_signals = []

# 4. Run the simulation loop
upstream_head = 5.0 # Constant source water level
for t in range(num_steps):
    # Turn pump on at t=5, off at t=80
    if 5 <= t < 80:
        control_signal = 1
    else:
        control_signal = 0
    control_signals.append(control_signal)
    
    # Steadily increase the downstream head to test the max_head limit
    # Max head is 20m, upstream is 5m, so pump stops when downstream reaches 25m
    downstream_head = 10.0 + (t * 0.2)
    downstream_heads.append(downstream_head)
    
    action = {
        'control_signal': control_signal,
        'upstream_head': upstream_head,
        'downstream_head': downstream_head
    }
    
    current_state = pump.step(action=action, dt=dt)
    history.append(current_state.copy())

print("Pump simulation complete.")

## Results and Visualization

The plots show the pump's `status` and `outflow`. The outflow remains constant at the `max_flow_rate` until the head difference (`downstream_head` - `upstream_head`) exceeds the pump's `max_head` of 20m, at which point the outflow drops to zero, even though the pump's control signal is still on.

In [ ]:
# Extract data from history
time = [i * dt for i in range(num_steps)]
statuses = [h['status'] for h in history]
outflows = [h['outflow'] for h in history]
power_draws = [h['power_draw_kw'] for h in history]

# Create a DataFrame
df = pd.DataFrame({
    'Time': time,
    'Control Signal': control_signals,
    'Outflow': outflows,
    'Power Draw (kW)': power_draws,
    'Downstream Head': downstream_heads
})

print(df.head())

# Plot the results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# Outflow and Head
ax1.plot(df['Time'], df['Outflow'], label='Pump Outflow', color='blue')
ax1_twin = ax1.twinx()
ax1_twin.plot(df['Time'], df['Downstream Head'], label='Downstream Head', color='gray', linestyle='--')
ax1.set_ylabel('Outflow (m^3/s)')
ax1_twin.set_ylabel('Head (m)')
ax1.set_title('Pump Performance vs. Head')
ax1.grid(True)
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax1_twin.get_legend_handles_labels()
ax1_twin.legend(lines + lines2, labels + labels2, loc='upper right')

# Status and Power
ax2.plot(df['Time'], df['Control Signal'], label='Control Signal (1=On)', drawstyle='steps-post', linestyle='--')
ax2.plot(df['Time'], df['Power Draw (kW)'], label='Power Draw (kW)', color='red')
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Power (kW) / Status')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()